<a href="https://colab.research.google.com/gist/maclandrol/12eed4ad1df93c712325cd5972447f01/03_Texte_Medical_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03. Analyse de Textes Médicaux avec IA

**Enseignant:** Emmanuel Noutahi, PhD

---

**Objectif:** Utiliser l'IA pour analyser automatiquement des comptes-rendus médicaux.

**Applications pratiques :**
- Triage automatique de comptes-rendus
- Classification par niveau d'urgence
- Extraction d'informations clés
- Détection de pathologies dans le texte

**Important:** Ce cours vous montre comment utiliser des outils IA existants, pas comment les créer.

## Installation et Configuration

In [ ]:
# Installation des bibliothèques pour l'analyse de texte médical
!pip install scikit-learn pandas numpy matplotlib seaborn -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
import re
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("✅ Bibliothèques installées et configurées avec succès")

## Création du Dataset de Comptes-Rendus

In [ ]:
# Créer un dataset simulé de comptes-rendus médicaux
comptes_rendus = [
    {
        "texte": "Patient présente une légère toux sèche depuis 3 jours. Examen pulmonaire normal. Auscultation cardiaque normale. Température 37.2°C. Prescription d'un sirop antitussif. Retour à domicile avec surveillance.",
        "urgence": "faible"
    },
    {
        "texte": "Douleur thoracique aiguë survenue brutalement. ECG montre des anomalies du segment ST. Troponines élevées. Suspicion d'infarctus du myocarde. Orientation immédiate en cardiologie.",
        "urgence": "haute"
    },
    {
        "texte": "Contrôle post-opératoire après appendicectomie. Cicatrisation correcte, pas de signes infectieux. Patient apyrétique. Ablation des fils prévue dans 7 jours. Évolution favorable.",
        "urgence": "faible"
    },
    {
        "texte": "Détresse respiratoire sévère. SpO2 à 85% en air ambiant. Râles crépitants bilatéraux. Suspicion d'œdème aigu pulmonaire. Oxygénothérapie urgente et diurétiques IV.",
        "urgence": "haute"
    },
    {
        "texte": "Hypertension artérielle modérément déséquilibrée. TA 160/95 mmHg. Ajustement du traitement antihypertenseur. Contrôle recommandé dans 3 mois avec bilan lipidique.",
        "urgence": "moyenne"
    },
    {
        "texte": "Consultation de routine pour diabète type 2. Glycémie à jeun 1.25 g/L. HbA1c à 7.2%. Adaptation posologique de la metformine. Suivi diététique renforcé.",
        "urgence": "moyenne"
    },
    {
        "texte": "AVC ischémique aigu confirmé par IRM. Déficit moteur du côté droit. Prise en charge en unité neurovasculaire. Thrombolyse envisagée si dans les délais.",
        "urgence": "haute"
    },
    {
        "texte": "Gastroentérite probablement virale. Diarrhées liquides depuis 24h. Pas de fièvre. Traitement symptomatique et réhydratation orale. Surveillance à domicile.",
        "urgence": "faible"
    },
    {
        "texte": "Crise d'asthme modérée. Dyspnée d'effort. Peak-flow diminué à 60% de la théorique. Traitement bronchodilatateur renforcé. Contrôle pneumologique à prévoir.",
        "urgence": "moyenne"
    },
    {
        "texte": "Choc anaphylactique suite à prise d'antibiotique. Urticaire généralisé et hypotension. Adrénaline intramusculaire et corticoïdes IV. Surveillance en réanimation.",
        "urgence": "haute"
    },
    {
        "texte": "Migraine habituelle chez patiente connue. Céphalées pulsatiles sans signes neurologiques. Traitement antalgique habituel efficace. Pas de signe d'alarme.",
        "urgence": "faible"
    },
    {
        "texte": "Fracture du poignet droit suite à chute. Radiographie confirme fracture de Colles. Réduction sous anesthésie locale. Immobilisation plâtrée pour 6 semaines.",
        "urgence": "moyenne"
    },
    {
        "texte": "État de mal épileptique. Crises généralisées tonico-cloniques répétées. Traitement antiépileptique IV urgent. Transfert en réanimation neurologique.",
        "urgence": "haute"
    },
    {
        "texte": "Consultation préventive. Bilan de santé normal. Vaccinations à jour. Conseils hygiéno-diététiques. Prochaine consultation dans 1 an.",
        "urgence": "faible"
    },
    {
        "texte": "Insuffisance cardiaque décompensée. Dyspnée de repos, œdèmes des membres inférieurs. BNP élevé. Adaptation du traitement diurétique et surveillance.",
        "urgence": "haute"
    },
    {
        "texte": "Lombalgie commune sans signes de gravité. Pas de déficit neurologique. Traitement antalgique et anti-inflammatoire. Repos relatif conseillé.",
        "urgence": "faible"
    },
    {
        "texte": "Pneumonie communautaire probable. Fièvre 39°C, toux productive. Radiographie pulmonaire montre un foyer infectieux. Antibiothérapie débutée.",
        "urgence": "moyenne"
    },
    {
        "texte": "Hémorragie digestive haute. Hématémèse et méléna. Choc hémorragique. Transfusion sanguine urgente et endoscopie en urgence.",
        "urgence": "haute"
    },
    {
        "texte": "Anxiété généralisée chez patient stressé. Pas de signes organiques. Traitement anxiolytique ponctuel. Orientation vers psychologue recommandée.",
        "urgence": "faible"
    },
    {
        "texte": "Calcul rénal avec colique néphrétique. Douleur intense lombaire droite. Scanner confirme lithiase urétérale. Traitement antalgique et surveillance.",
        "urgence": "moyenne"
    }
]

# Convertir en DataFrame
df = pd.DataFrame(comptes_rendus)
print(f"📊 Dataset créé: {len(df)} comptes-rendus médicaux")
print(f"📋 Aperçu des données:")
df.head()

## Exploration du Dataset

In [ ]:
# Analyse de la répartition des niveaux d'urgence
print("🏥 RÉPARTITION DES NIVEAUX D'URGENCE:")
print("=" * 40)
urgence_counts = df['urgence'].value_counts()
print(urgence_counts)

print(f"\n📊 Pourcentages:")
for urgence, count in urgence_counts.items():
    pourcentage = (count / len(df)) * 100
    print(f"• {urgence.capitalize()}: {count} cas ({pourcentage:.1f}%)")

# Longueur des textes
df['longueur_texte'] = df['texte'].apply(len)
print(f"\n📝 STATISTIQUES DES TEXTES:")
print(f"• Longueur moyenne: {df['longueur_texte'].mean():.0f} caractères")
print(f"• Longueur médiane: {df['longueur_texte'].median():.0f} caractères")
print(f"• Longueur min/max: {df['longueur_texte'].min()}/{df['longueur_texte'].max()} caractères")

In [ ]:
# Visualisations de l'exploration
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Graphique 1: Répartition des urgences
urgence_counts.plot(kind='pie', ax=axes[0], autopct='%1.1f%%', 
                   colors=['lightgreen', 'orange', 'salmon'])
axes[0].set_title('Répartition des Niveaux d\'Urgence')
axes[0].set_ylabel('')

# Graphique 2: Longueur des textes par urgence
for urgence in df['urgence'].unique():
    subset = df[df['urgence'] == urgence]['longueur_texte']
    axes[1].hist(subset, alpha=0.7, label=urgence.capitalize(), bins=8)
axes[1].set_title('Distribution de la Longueur des Textes par Urgence')
axes[1].set_xlabel('Longueur (caractères)')
axes[1].set_ylabel('Fréquence')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Analyse des Mots-Clés Médicaux

In [ ]:
# Analyse des mots les plus fréquents par niveau d'urgence
def analyser_mots_cles(textes, urgence_niveau):
    """
    Analyser les mots-clés les plus fréquents pour un niveau d'urgence
    """
    # Joindre tous les textes de ce niveau
    texte_complet = ' '.join(textes).lower()
    
    # Nettoyer le texte
    texte_propre = re.sub(r'[^a-zàâäéèêëïîôöùûüç\s]', ' ', texte_complet)
    
    # Mots vides médicaux français
    mots_vides = ['le', 'la', 'les', 'un', 'une', 'des', 'du', 'de', 'et', 'ou', 
                  'avec', 'sans', 'dans', 'sur', 'pour', 'par', 'depuis', 'après',
                  'patient', 'patiente', 'cas', 'suite']
    
    # Compter les mots
    mots = texte_propre.split()
    compteur_mots = {}
    
    for mot in mots:
        if len(mot) > 3 and mot not in mots_vides:
            compteur_mots[mot] = compteur_mots.get(mot, 0) + 1
    
    # Retourner les 10 plus fréquents
    return sorted(compteur_mots.items(), key=lambda x: x[1], reverse=True)[:10]

print("🔍 MOTS-CLÉS LES PLUS FRÉQUENTS PAR URGENCE:")
print("=" * 50)

for urgence in ['faible', 'moyenne', 'haute']:
    textes_urgence = df[df['urgence'] == urgence]['texte'].tolist()
    mots_cles = analyser_mots_cles(textes_urgence, urgence)
    
    print(f"\n📌 Urgence {urgence.upper()}:")
    for i, (mot, freq) in enumerate(mots_cles[:8], 1):
        print(f"  {i}. {mot} (×{freq})")

## Construction du Modèle de Classification

In [ ]:
# Préparation des données pour l'apprentissage automatique
print("🔧 PRÉPARATION DES DONNÉES:")
print("=" * 30)

# Encoder les catégories d'urgence
label_map = {'faible': 0, 'moyenne': 1, 'haute': 2}
label_inverse = {0: 'faible', 1: 'moyenne', 2: 'haute'}

# Préparer les données
textes = df['texte'].tolist()
labels = [label_map[urgence] for urgence in df['urgence']]

print(f"Nombre de comptes-rendus: {len(textes)}")
print(f"Répartition des classes:")
for urgence, count in df['urgence'].value_counts().items():
    print(f"  • {urgence}: {count} cas")

# Vectorisation TF-IDF pour transformer le texte en nombres
print(f"\n📊 VECTORISATION TF-IDF:")
vectorizer = TfidfVectorizer(
    max_features=200,     # Maximum 200 mots les plus importants
    stop_words=None,      # Pas de mots vides automatiques (on gère manuellement)
    lowercase=True,       # Convertir en minuscules
    ngram_range=(1, 2)    # Mots individuels et bigrammes
)

X = vectorizer.fit_transform(textes)
y = np.array(labels)

print(f"Matrice de caractéristiques: {X.shape}")
print(f"  • {X.shape[0]} documents")
print(f"  • {X.shape[1]} caractéristiques (mots/phrases)")

In [ ]:
# Division train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"📊 DIVISION DES DONNÉES:")
print(f"Données d'entraînement: {X_train.shape[0]} comptes-rendus")
print(f"Données de test: {X_test.shape[0]} comptes-rendus")

# Entraînement de plusieurs modèles
modeles = {
    'Régression Logistique': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100)
}

resultats = {}

print(f"\n🌳 ENTRAÎNEMENT DES MODÈLES:")
print("=" * 40)

for nom_modele, modele in modeles.items():
    # Entraînement
    modele.fit(X_train, y_train)
    
    # Prédictions
    y_pred = modele.predict(X_test)
    
    # Métriques
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    resultats[nom_modele] = {
        'modele': modele,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'predictions': y_pred
    }
    
    print(f"\n{nom_modele}:")
    print(f"  • Précision: {accuracy:.3f} ({accuracy*100:.1f}%)")
    print(f"  • Précision pondérée: {precision:.3f}")
    print(f"  • Sensibilité pondérée: {recall:.3f}")
    print(f"  • Score F1 pondéré: {f1:.3f}")

# Sélectionner le meilleur modèle
meilleur_modele_nom = max(resultats.keys(), key=lambda x: resultats[x]['accuracy'])
meilleur_modele = resultats[meilleur_modele_nom]['modele']

print(f"\n🏆 Meilleur modèle: {meilleur_modele_nom}")
print(f"    Précision: {resultats[meilleur_modele_nom]['accuracy']*100:.1f}%")

## 📊 Évaluation Détaillée du Modèle

In [ ]:
# Rapport de classification détaillé pour le meilleur modèle
y_pred_best = resultats[meilleur_modele_nom]['predictions']

print(f"📋 RAPPORT DÉTAILLÉ - {meilleur_modele_nom.upper()}:")
print("=" * 50)
print(classification_report(y_test, y_pred_best, 
                          target_names=['Urgence Faible', 'Urgence Moyenne', 'Urgence Haute']))

# Matrice de confusion
cm = confusion_matrix(y_test, y_pred_best)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Faible', 'Moyenne', 'Haute'],
            yticklabels=['Faible', 'Moyenne', 'Haute'])
plt.title(f'Matrice de Confusion - {meilleur_modele_nom}')
plt.ylabel('Vraie Urgence')
plt.xlabel('Urgence Prédite')
plt.show()

# Analyse des erreurs
print(f"\n🔍 ANALYSE DES ERREURS:")
erreurs = np.where(y_test != y_pred_best)[0]
print(f"Nombre d'erreurs: {len(erreurs)} sur {len(y_test)} prédictions")

if len(erreurs) > 0:
    print(f"\nExemples d'erreurs:")
    test_indices = np.arange(len(y_test))
    for i, erreur_idx in enumerate(erreurs[:3]):
        original_idx = test_indices[erreur_idx]
        vraie_urgence = label_inverse[y_test[erreur_idx]]
        pred_urgence = label_inverse[y_pred_best[erreur_idx]]
        # Retrouver le texte original
        texte_erreur = df.iloc[original_idx]['texte'][:100] + "..."
        print(f"  {i+1}. Prédit '{pred_urgence}' au lieu de '{vraie_urgence}'")
        print(f"     Texte: {texte_erreur}")

## 🎯 Mots les Plus Importants pour la Classification

In [ ]:
# Analyser les mots les plus importants (si modèle linéaire)
if meilleur_modele_nom == 'Régression Logistique':
    print("🎯 MOTS LES PLUS DISCRIMINANTS:")
    print("=" * 40)
    
    feature_names = vectorizer.get_feature_names_out()
    
    for classe_idx, classe_nom in enumerate(['Faible', 'Moyenne', 'Haute']):
        # Coefficients pour cette classe
        coefficients = meilleur_modele.coef_[classe_idx]
        
        # Top mots positifs (indiquent cette classe)
        top_positifs_idx = np.argsort(coefficients)[-8:][::-1]
        
        # Top mots négatifs (indiquent l'absence de cette classe)
        top_negatifs_idx = np.argsort(coefficients)[:5]
        
        print(f"\n📌 URGENCE {classe_nom.upper()}:")
        print(f"  Mots indicateurs (+):")
        for idx in top_positifs_idx:
            if coefficients[idx] > 0:
                print(f"    • {feature_names[idx]} ({coefficients[idx]:.3f})")
        
        print(f"  Mots contre-indicateurs (-):")
        for idx in top_negatifs_idx:
            if coefficients[idx] < 0:
                print(f"    • {feature_names[idx]} ({coefficients[idx]:.3f})")

elif meilleur_modele_nom == 'Random Forest':
    print("🎯 CARACTÉRISTIQUES LES PLUS IMPORTANTES (Random Forest):")
    print("=" * 50)
    
    feature_names = vectorizer.get_feature_names_out()
    importances = meilleur_modele.feature_importances_
    
    # Top 15 caractéristiques les plus importantes
    top_indices = np.argsort(importances)[-15:][::-1]
    
    for i, idx in enumerate(top_indices, 1):
        print(f"  {i:2d}. {feature_names[idx]:<20} : {importances[idx]:.4f}")

## 🔮 Prédiction sur Nouveaux Comptes-Rendus

In [ ]:
# Fonction pour analyser un nouveau compte-rendu
def analyser_compte_rendu(texte_nouveau):
    """
    Analyser l'urgence d'un nouveau compte-rendu médical
    """
    # Transformer le texte avec le même vectorizer
    texte_vectorise = vectorizer.transform([texte_nouveau])
    
    # Prédire l'urgence
    prediction = meilleur_modele.predict(texte_vectorise)[0]
    probabilites = meilleur_modele.predict_proba(texte_vectorise)[0]
    
    urgence_predite = label_inverse[prediction]
    confiance = max(probabilites) * 100
    
    # Déterminer le niveau d'alerte
    if urgence_predite == 'haute':
        emoji = "🚨"
        action = "INTERVENTION URGENTE"
    elif urgence_predite == 'moyenne':
        emoji = "⚠️"
        action = "Surveillance renforcée"
    else:
        emoji = "✅"
        action = "Suivi de routine"
    
    return {
        'urgence': urgence_predite,
        'confiance': confiance,
        'probabilites': {
            'faible': probabilites[0] * 100,
            'moyenne': probabilites[1] * 100,
            'haute': probabilites[2] * 100
        },
        'emoji': emoji,
        'action': action
    }

# Tester avec de nouveaux exemples
nouveaux_cas = [
    "Arrêt cardiorespiratoire. Massage cardiaque en cours. Défibrillation nécessaire. Réanimation en urgence absolue.",
    "Rhume banal avec écoulement nasal. Pas de fièvre. Traitement symptomatique. Amélioration attendue.",
    "Phlébite du mollet gauche confirmée à l'échodoppler. Anticoagulation débutée. Surveillance biologique.",
    "Plaie superficielle au genou après chute. Désinfection et pansement. Contrôle dans 48 heures."
]

print("👥 ANALYSE DE NOUVEAUX COMPTES-RENDUS:")
print("=" * 50)

for i, cas in enumerate(nouveaux_cas, 1):
    resultat = analyser_compte_rendu(cas)
    
    print(f"\n{resultat['emoji']} CAS {i}:")
    print(f"Texte: {cas[:80]}...")
    print(f"Urgence prédite: {resultat['urgence'].upper()}")
    print(f"Confiance: {resultat['confiance']:.1f}%")
    print(f"Action recommandée: {resultat['action']}")
    print(f"Probabilités détaillées:")
    for niveau, prob in resultat['probabilites'].items():
        print(f"  • {niveau}: {prob:.1f}%")

## 🎯 Outil Pratique de Triage Automatique

In [ ]:
# Outil complet de triage automatique
def triage_automatique(comptes_rendus_liste):
    """
    Effectuer un triage automatique sur une liste de comptes-rendus
    
    Args:
        comptes_rendus_liste: Liste de dictionnaires avec 'id' et 'texte'
    
    Returns:
        Liste triée par ordre de priorité
    """
    resultats = []
    
    for cr in comptes_rendus_liste:
        analyse = analyser_compte_rendu(cr['texte'])
        
        # Score de priorité (urgence haute = 3, moyenne = 2, faible = 1)
        score_priorite = {'haute': 3, 'moyenne': 2, 'faible': 1}[analyse['urgence']]
        
        resultats.append({
            'id': cr['id'],
            'texte': cr['texte'][:100] + '...',
            'urgence': analyse['urgence'],
            'confiance': analyse['confiance'],
            'emoji': analyse['emoji'],
            'action': analyse['action'],
            'score_priorite': score_priorite
        })
    
    # Trier par score de priorité décroissant, puis par confiance
    resultats_tries = sorted(resultats, 
                           key=lambda x: (x['score_priorite'], x['confiance']), 
                           reverse=True)
    
    return resultats_tries

# Exemple d'utilisation
exemple_urgences = [
    {
        'id': 'CR001',
        'texte': 'Contrôle post-vaccination. Pas d\'effets secondaires. État général excellent.'
    },
    {
        'id': 'CR002', 
        'texte': 'Embolie pulmonaire massive. Détresse respiratoire majeure. Anticoagulation en urgence.'
    },
    {
        'id': 'CR003',
        'texte': 'Entorse de cheville grade 1. Douleur modérée. Traitement antalgique et repos.'
    },
    {
        'id': 'CR004',
        'texte': 'Coma diabétique. Glycémie à 5g/L. Perfusion insuline et réanimation médicale.'
    },
    {
        'id': 'CR005',
        'texte': 'Otite moyenne aiguë. Tympan rouge et bombé. Antibiothérapie prescrite.'
    }
]

triage_resultat = triage_automatique(exemple_urgences)

print("🏥 RÉSULTAT DU TRIAGE AUTOMATIQUE:")
print("=" * 50)
print("Ordre de priorité (du plus urgent au moins urgent):\n")

for i, resultat in enumerate(triage_resultat, 1):
    print(f"{i}. {resultat['emoji']} {resultat['id']} - Urgence {resultat['urgence'].upper()}")
    print(f"   Confiance: {resultat['confiance']:.1f}%")
    print(f"   Action: {resultat['action']}")
    print(f"   Texte: {resultat['texte']}")
    print()

# Statistiques du triage
stats_triage = {'haute': 0, 'moyenne': 0, 'faible': 0}
for resultat in triage_resultat:
    stats_triage[resultat['urgence']] += 1

print("📊 STATISTIQUES DU TRIAGE:")
print(f"🚨 Urgence haute: {stats_triage['haute']} cas")
print(f"⚠️ Urgence moyenne: {stats_triage['moyenne']} cas")
print(f"✅ Urgence faible: {stats_triage['faible']} cas")

## ✅ Résumé et Applications

In [ ]:
# Résumé final de l'analyse
print("🏥 RÉSUMÉ DE L'ANALYSE DE TEXTES MÉDICAUX:")
print("=" * 50)
print(f"📊 Dataset analysé: {len(df)} comptes-rendus médicaux")
print(f"🤖 Meilleur modèle: {meilleur_modele_nom}")
print(f"🎯 Performance: {resultats[meilleur_modele_nom]['accuracy']*100:.1f}% de précision")
print(f"📋 Classes: 3 niveaux d'urgence (faible, moyenne, haute)")

print(f"\n💡 APPLICATIONS PRATIQUES IMMÉDIATES:")
print(f"✅ Triage automatique des comptes-rendus entrants")
print(f"✅ Priorisation des consultations selon l'urgence")
print(f"✅ Alerte automatique pour les cas critiques")
print(f"✅ Aide à la décision pour les médecins régulateurs")
print(f"✅ Optimisation des flux dans les services d'urgence")
print(f"✅ Surveillance de la qualité des comptes-rendus")

print(f"\n🔧 OUTILS DÉVELOPPÉS:")
print(f"📝 Analyseur de compte-rendu individuel")
print(f"🏥 Système de triage automatique par lots")
print(f"📊 Tableau de bord des urgences")
print(f"🎯 Extraction des mots-clés discriminants")

print(f"\n⚠️ LIMITATIONS IMPORTANTES:")
print(f"• Dataset d'entraînement limité (pour démonstration)")
print(f"• Nécessite validation sur de vraies données hospitalières")
print(f"• Performance dépend de la qualité de rédaction")
print(f"• Ne remplace pas l'expertise médicale humaine")
print(f"• Doit être adapté au vocabulaire de chaque service")

print(f"\n🎓 COMPÉTENCES ACQUISES:")
print(f"✅ Préparation de données textuelles médicales")
print(f"✅ Vectorisation TF-IDF pour l'analyse de texte")
print(f"✅ Classification multiclasse avec scikit-learn")
print(f"✅ Évaluation de modèles de classification de texte")
print(f"✅ Analyse des mots-clés discriminants")
print(f"✅ Développement d'outils de triage automatique")

print(f"\n🎯 PROCHAINE ÉTAPE:")
print(f"Dans le prochain notebook, nous utiliserons PyTorch pour des analyses plus avancées!")

# Afficher un exemple de template final
print(f"\n💡 TEMPLATE PRÊT À UTILISER:")
print(f"```python")
print(f"# Pour analyser un nouveau compte-rendu:")
print(f"nouveau_texte = 'Votre compte-rendu ici...'")
print(f"resultat = analyser_compte_rendu(nouveau_texte)")
print(f"print(f'Urgence: {{resultat[\"urgence\"]}} ({{resultat[\"confiance\"]:.1f}}%)')")
print(f"```")